In [ ]:
import gzip
import json

import tqdm
import numpy as np
import tensorflow as tf
import tensorflow_gnn as tfgnn
import tensorflow_ranking as tfr

from tpu_graphs.baselines.layout.data import get_npz_dataset
from tpu_graphs.baselines.layout.models import ResModel
from gfos.metrics import kendall


In [ ]:
SOURCE = "xla"
SEARCH = "random"

# nlp random  aaa7c9876d5bde19db56594f7334657c
# xla random  801628c441d4b633a0fe36b72248f8e5
# xla default 5260f25ba9d0eae9c5e563a16848fd08

RUN_ID = "f50342e14012adf9da4ad3c0b046f0a6"
data_root_dir = rf"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout\{SOURCE}\{SEARCH}"
MODEL_DIR = f"../../src/tpu_graphs/output/model_{RUN_ID}"
RUN_FILE = f"../../src/tpu_graphs/output/run_{RUN_ID}.jsonz"
_INFERENCE_CONFIGS_BATCH_SIZE = 50

num_configs = 16
max_configs = 3000


In [ ]:
# args = json.load(gzip.open(RUN_FILE))
# print(args["args"])


In [ ]:
# Need to comment out _TOY_DATA in get_npz_split

with tf.device("/CPU:0"):
    # Load whole dataset to get `num_ops`
    dataset = get_npz_dataset(
        data_root_dir,
        min_train_configs=-1,
        max_train_configs=max_configs,
        cache_dir=None,
    )

    num_ops = dataset.num_ops
    print(f"num_ops: {num_ops}")
    # del dataset

    # Load whole valid data will raise error:
    # Cannot create a tensor proto whose content is larger than 2GB.
    # valid_split = get_npz_split(
    #     data_root_dir + r"\valid",
    #     max_configs=1000,
    #     cache_dir=None,
    # )

In [ ]:
model = ResModel(num_configs, num_ops)

for graph in dataset.validation.iter_graph_tensors():
    num_configs = graph.node_sets["g"]["runtimes"].shape[-1]

    for i in range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE):
        end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
        # Take a cut of the configs.
        node_set_g = graph.node_sets["g"]
        subconfigs_graph = tfgnn.GraphTensor.from_pieces(
            edge_sets=graph.edge_sets,
            node_sets={
                "op": graph.node_sets["op"],
                "nconfig": tfgnn.NodeSet.from_fields(
                    sizes=graph.node_sets["nconfig"].sizes,
                    features={
                        "feats": graph.node_sets["nconfig"]["feats"][
                            :, i:end_i
                        ],
                    },
                ),
                "g": tfgnn.NodeSet.from_fields(
                    sizes=tf.constant([1]),
                    features={
                        "graph_id": node_set_g["graph_id"],
                        "runtimes": node_set_g["runtimes"][:, i:end_i],
                        "kept_node_ratio": node_set_g["kept_node_ratio"],
                    },
                ),
            },
        )
        h = model.forward(
            subconfigs_graph, num_configs=end_i-i, backprop=False
        )
        break
    break



In [ ]:
loaded = tf.keras.models.load_model(
    MODEL_DIR,
    compile=False,
    custom_objects={"opa_metric": tfr.keras.metrics.OPAMetric},
)

for v, lv in zip(model.trainable_variables, loaded.trainable_variables):
    v.assign(lv.value())


In [ ]:
# xla random
# loaded = tf.keras.models.load_model(
#     "../../src/tpu_graphs/output/model_801628c441d4b633a0fe36b72248f8e5/",
#     custom_objects={"opa_metric": tfr.keras.metrics.OPAMetric},
# )

# xla default
# loaded = tf.keras.models.load_model(
#     "../../src/tpu_graphs/output/model_5260f25ba9d0eae9c5e563a16848fd08",
#     custom_objects={"opa_metric": tfr.keras.metrics.OPAMetric},
# )


In [ ]:
scores = []

num_files = dataset.validation.graph_id.shape[-1]
for i, graph in enumerate(dataset.validation.iter_graph_tensors()):
    graph_id = graph.node_sets["g"]["graph_id"][0].numpy().decode()
    num_configs = graph.node_sets["g"]["runtimes"].shape[-1]
    outs = []

    for i in tqdm.tqdm(
        range(0, num_configs, _INFERENCE_CONFIGS_BATCH_SIZE),
        total=int(np.ceil(num_configs / _INFERENCE_CONFIGS_BATCH_SIZE)),
        desc=f"[{i+1}/{num_files}] {graph_id}",
        leave=False,
    ):
        end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, num_configs)
        # Take a cut of the configs.
        node_set_g = graph.node_sets["g"]
        subconfigs_graph = tfgnn.GraphTensor.from_pieces(
            edge_sets=graph.edge_sets,
            node_sets={
                "op": graph.node_sets["op"],
                "nconfig": tfgnn.NodeSet.from_fields(
                    sizes=graph.node_sets["nconfig"].sizes,
                    features={
                        "feats": graph.node_sets["nconfig"]["feats"][
                            :, i:end_i
                        ],
                    },
                ),
                "g": tfgnn.NodeSet.from_fields(
                    sizes=tf.constant([1]),
                    features={
                        "graph_id": node_set_g["graph_id"],
                        "runtimes": node_set_g["runtimes"][:, i:end_i],
                        "kept_node_ratio": node_set_g["kept_node_ratio"],
                    },
                ),
            },
        )
        h = model.forward(
            subconfigs_graph, num_configs=end_i-i, backprop=False
        )
        outs.append(h[0])

    outs = tf.concat(outs, axis=0)
    sorted_indices = np.argsort(outs.numpy().squeeze())
    true_indices = np.argsort(graph.node_sets["g"]["runtimes"].numpy().squeeze())

    scores.append(kendall(true_indices, sorted_indices))


In [ ]:
np.mean(scores)

- nlp random: 0.5120780780780779  [5260f25ba9d0eae9c5e563a16848fd08]
- xla random: 0.10891880891880891 [f50342e14012adf9da4ad3c0b046f0a6]